<h1 style="text-align: center;">Analyses d'un carrousel</h1>

<h2 style="text-align: center;">Date: À REMPLIR</h2>

<h2 style="text-align: center;">Contexte: À REMPLIR</h2>

# Préparation

## Imports des modules et configuration de base

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
#os.sys.path.append('')

### # import du package `carrouselsAnalysis` et des classes, et tracks préconfigurées 

In [ ]:
# on importe le package complet `carrouselsAnalysis`
import carrouselsAnalysis

In [ ]:
# on importe les classes Track et Carrousel du module `data`
from carrouselsAnalysis.carrousel import Track, Carrousel

**NOTE pour l'utilisateur**:
- si nécessaire, changer le nom de la track préconfigurée à importer ci-dessous, après la mention `import`

In [ ]:
# on importe la configuration de track LilleL1 pré-configurée dans le module `tracks`
from carrouselsAnalysis.tracks import LilleL1

# Création de la track et d'une instance de carrousel

**NOTE pour l'utilisateur**:
- adapter ci-dessous le chemin des données `chemin_data`, le format des données `format_data` ainsi que les éléments `source` et `contexte` du carrousel
- ajouter une référence (textuelle) de `build` s'il ne s'agit pas d'une version officielle

In [ ]:
# chemin des données à importer en local
chemin_data = '../carrousels/data/site/Lille_FLUENCE_DATA-2022-10-16&17__v1.xlsx'
format_data = 'MG'

# source des données. Typiquement 'FIVP' ou 'Site'
source = 'Site'

# contexte du carrousel. Typiquement 'mono-train' ou 'multi-trains'
contexte = 'multi-trains'

# référence du build utilisé (si non officiel, sinon la SoftwareVersion utilisée est celle extraite de l'OMAP)
build = '137b65'

**NOTE pour l'utilisateur**:
- changer la référence de la track préconfigurée utilisée ci-dessous si ce n'est pas LilleL1

In [ ]:
# création de la track LilleL1
track_1 = Track(LilleL1)

In [ ]:
# création du carrousel
c = Carrousel(track=track_1, source=source, context=contexte, dataPath=chemin_data, dataFormat=format_data, build=build)

## Import des données

utilisation des méthodes `get_movement`, `get_EB`

In [ ]:
# on importe les données de mouvement
c.get_movement()

TODO METHOD `get_movement`:
- utiliser la colonne `SW version` pour définir un attribut de classe `version` qui pourra être hérité

In [ ]:
# on importe les données de freinage d'urgence
c.get_EB()

TODO METHOD `get_EB`:
- récupérer la `version` de la classe carrousel

# Graphiques & indicateurs

## Précision d'arrêt en station

Rappel des objectifs contractuels:
- $99\%$ des arrêts à moins de 17cm
- $1/100000$ arrêt raté

Rappel des objectifs Alstom pour MeS 26m:
- $99\%$ des arrêts à moins de **30cm**
- $1/100 000$ arrêt raté

### Au global

In [ ]:
c.histo_precision(xRange=(-1.3, 1.3))

### Histogramme comparé en fonction du Docking Status

In [ ]:
c.histo_precision_compare(dataCompare='CorrectDocking', y='freq', style='barstacked')

### Histogramme cumulé par tranche de précision

In [ ]:
c.histo_precision_compare(x='absoluteDistanceSSP', dataCompare='CorrectDocking', y='freq', style='barstacked',
                         cumulative=True, xlim=(-0.1, 1.5), bins=[0, 0.17, 0.3, 0.5, 0.6, 0.82, 1.5])

### Histogramme partiels

#### en ligne réceptive

In [ ]:
c.histo_precision_filter(y='freq', dataFilter='resistivityStateDual', filterValue='receptive')

#### en ligne non réceptive

In [ ]:
c.histo_precision_filter(y='freq', dataFilter='resistivityStateDual', filterValue='non-receptive')

In [ ]:
c.histo_precision_filter(x='absoluteDistanceSSP', y='freq', cumulative=True, xlim=(-0.1, 1.5), bins=[0, 0.17, 0.3, 0.5, 0.6, 0.82, 1.5],
                         dataFilter='resistivityStateDetailed', filterValue='non-receptive', legend=True)

## Arrêts ratés

### Synthèse

In [ ]:
c.synthese_arrets_rates(detailedState=True)

#### répartition statistique des arrêts ratés

In [ ]:
c.histo_precision_filter(x='absoluteDistanceSSP', dataFilter='CorrectDocking', filterValue=False, cumulative=True, y='freq', xlim=(-0.1, 1.5),
                        bins=[0, 0.17, 0.3, 0.5, 0.6, 0.82, 1.5], title="Répartition statistique des arrêts en station ratés")

### En ligne réceptive

In [ ]:
c.camembert_arrets_rates(resistivityState='receptive')

### En ligne non réceptive

In [ ]:
c.camembert_arrets_rates(resistivityState=['non-receptive', 'partiellement'])

### Par station

In [ ]:
c.missed_stops_pct_by_station()

### Par train

In [ ]:
c.missed_stops_pct_by_train(cat='Train')

### Par période de temps

## Précision d'arrêt par station

### Précision moyenne

**Note pour l'utilisateur**:
- pour ajouter un intervalle de confiance basé sur l'écart type, utiliser `ci='sd'`

In [ ]:
c.trace_precision_station()

### Dispersion des arrêts par station

In [ ]:
c.trace_dispersion_station(xlim=(1.5, -1.5))

## Précision d'arrêt par train

### Précision moyenne par train

In [ ]:
c.trace_precision_train()

### Dispersion par train

In [ ]:
c.trace_dispersion_train(figsize=(9.5,12), xlim=(1.5, -1.5))

## Temps de parcours

### Temps de parcours moyen / mini / maxi

In [ ]:
c.trace_tps_parcours()

### Dispersion des temps de parcours

In [ ]:
c.trace_disp_tps_parcours(ylim=(40,110))

## Temps de parcours comparés mono-train vs multi-trains

À FAIRE

## Vitesse commerciale

À FAIRE: utilisation des `PexMovements` de la track

### Vitesse commerciale moyenne par voie

### Vitesse commerciale avec les temps de parcours mini

## Freinages d'urgence

### Répartition des freinages d'urgence par mouvement

#### y compris par type de mouvement (HLP, PEX, ??)

### Répartition des freinages d'urgence par KP

In [ ]:
c.trace_EB_by_KP(ylim=(0,15))

### Répartition des freinages d'urgence dans le temps

Note TODO:
- améliorer la représentation du format date dans les label d'axe
- permettre de choisir la période de temps représentée

In [ ]:
c.trace_EB_by_time(bins='30T', ylim=(0,15))

### Répartition des freinages d'urgence par cause d'EB

À FAIRE

## Corrélations

### Corrélation entre nombre de FU par interstation et écart entre temps de parcours moyen et mini